In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
#need Ngl for Charlie's function to work, but can't seem to import correctly. May need to change environment
#import Ngl
import esmlab

import warnings
warnings.filterwarnings("ignore")
#note: Dan uses "import glob" -- glob handles wildcards better than xarray for reading in multiple files

In [2]:
import intake

In [3]:
catalog = intake.open_esm_datastore("/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cmip6.json")
catalog.df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,time_range,path
0,AerChemMIP,BCC,BCC-ESM1,ssp370,r1i1p1f1,Amon,hfss,gn,NaN,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
1,AerChemMIP,BCC,BCC-ESM1,ssp370,r1i1p1f1,Amon,pr,gn,NaN,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
2,AerChemMIP,BCC,BCC-ESM1,ssp370,r1i1p1f1,Amon,rsus,gn,NaN,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
3,AerChemMIP,BCC,BCC-ESM1,ssp370,r1i1p1f1,Amon,tas,gn,NaN,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
4,AerChemMIP,BCC,BCC-ESM1,ssp370,r1i1p1f1,Amon,ts,gn,NaN,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...


In [ ]:
test_sr = catalog.search(experiment_id=['historical'], variable_id='co2')

In [ ]:
test_sr.unique('source_id')

In [ ]:
#don't really understand this output
test_sr.unique('member_id')

### Reading catalog to load CO2 values for CMIP6 models
Multiple configurations of BCC and CESM available. Choosing one narrows models to 4.
Difficult to tell how many models have multiple ensemble members, so only using first member

In [4]:
table_id = 'Amon'
var = 'co2'
# note that there are different numbers of model output available depending on options used for source_id and member_id
# should I consider using multiple ensemble members per model? multiple model configurations per model?
#member_id = ["r1i1p1f1","r2i1p1f1","r3i1p1f1","r4i1p1f1","r5i1p1f1","r6i1p1f1"]
# note that MIROC doesn't show up here because it's member_id = r1i2p1f1. Not sure why this is numbered differently
cat = catalog.search(experiment_id=['historical'], variable_id=var,
#                        source_id=test_sr.unique('source_id')['source_id']['values'], 
                        source_id=['CESM2','BCC-ESM1','MRI-ESM2-0','GFDL-ESM4'], 
#                        table_id=table_id, member_id =member_id)
                        table_id=table_id, member_id ="r1i1p1f1")

In [5]:
cat.unique('source_id')['source_id']['values']

['BCC-ESM1', 'CESM2', 'GFDL-ESM4', 'MRI-ESM2-0']

In [6]:
cat.unique('member_id')['member_id']['values']

['r1i1p1f1']

In [7]:
cat.unique('grid_label')

{'grid_label': {'count': 2, 'values': ['gn', 'gr1']}}

### Converting data to dictionary

In [8]:
co2_ds = cat.to_dataset_dict()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [9]:
co2_ds.keys()

dict_keys(['CMIP.MRI.MRI-ESM2-0.historical.Amon.gn', 'CMIP.NOAA-GFDL.GFDL-ESM4.historical.Amon.gr1', 'CMIP.BCC.BCC-ESM1.historical.Amon.gn', 'CMIP.NCAR.CESM2.historical.Amon.gn'])

#### Note: Not all output is available on CMIP6 archive, so ignoring next 4 code blocks for now
### Get the fixed variables output, only available for historical simulations (gridcell area, landfrac)


In [ ]:
sr_fx_ctrl_vars = catalog.search(table_id = ['fx'], source_id = cat.unique('source_id')['source_id']['values'], 
                                 member_id = cat.unique('member_id')['member_id']['values'], 
                                 experiment_id = 'historical', variable_id = ['areacella', 'sftlf'])

In [ ]:
sr_fx_ctrl_vars.unique('grid_label')

In [ ]:
fx_ds = sr_fx_ctrl_vars.to_dataset_dict()

In [ ]:
#what happened to BCC? Doesn't seem to have fx (also checked /glade/collections/cmip/CMIP6/CMIP/BCC and online archive)
fx_ds.keys()

###   
### Starting data analysis
###   

### Calculating amplitude for each year

In [10]:
for sim_name, data in co2_ds.items():
# set_coords based on M. Long's suggestion so that subtraction not applied to coordinate variables
# works on on datasets, but not on the dictionary.
    data    = data.set_coords(['time_bnds', 'lat_bnds', 'lon_bnds'])
#    datamax = data.resample(time="Y").max()
#    datamin = data.resample(time="Y").min()
#    co2amp  = datamax.co2 - datamin.co2
    co2amp  = data.resample(time="Y").max() - data.resample(time="Y").min()
    print(co2amp)
    co2_ds[sim_name] = co2amp

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 64, lon: 128, member_id: 1, plev: 19, time: 165)
Coordinates:
  * time       (time) datetime64[ns] 1850-12-31 1851-12-31 ... 2014-12-31
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(64, 2), meta=np.ndarray>
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(128, 2), meta=np.ndarray>
  * lat        (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 82.31 85.1 87.86
  * lon        (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
  * plev       (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 1e+03 500.0 100.0
  * member_id  (member_id) <U8 'r1i1p1f1'
Dimensions without coordinates: bnds
Data variables:
    co2        (time, member_id, plev, lat, lon) float32 dask.array<chunksize=(1, 1, 19, 64, 128), meta=np.ndarray>
<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 180, lon: 288, member_id: 1, plev: 19, time: 165)
Coordinates:
  * time       (time) object 1850-12-31 00:00:00 ... 2014-12-31 00:00:00
    lat_bnds   (lat, 

In [ ]:
# Note: This doesn't work due to calendar mismatches.
# See example here for reading in a dataset: https://geocat-examples.readthedocs.io/en/latest/gallery/XY/NCL_xy_18.html#sphx-glr-gallery-xy-ncl-xy-18-py
# Not sure how to do this for reading in a dictionary

#for amp_name, amp in co2_ds.items():
#    amp_stnd = amp.sel(time=slice('1850','2014')) - amp.sel(time=slice('1850'))

### Weighting functions from Matt Long
#### Grid cell area is not available for all models, so need to use sin weighting functions

In [ ]:
#From Matt Long:
def infer_lat_name(ds): 
    lat_names = ['latitude', 'lat']
    for n in lat_names:
        if n in ds:
            return n
    raise ValueError('could not determine lat name')    


def infer_lon_name(ds):
    lon_names = ['longitude', 'lon']
    for n in lon_names:
        if n in ds:
            return n
    raise ValueError('could not determine lon name')

In [ ]:
#Also from Matt Long:
def lat_weights_regular_grid(lat):
    """
    Generate latitude weights for equally spaced (regular) global grids.
    Weights are computed as sin(lat+dlat/2)-sin(lat-dlat/2) and sum to 2.0.
    """  
    dlat = np.abs(np.diff(lat))
    np.testing.assert_almost_equal(dlat, dlat[0])
    w = np.abs(np.sin(np.radians(lat + dlat[0] / 2.)) - np.sin(np.radians(lat - dlat[0] / 2.)))

    if np.abs(lat[0]) > 89.9999:
        w[0] = np.abs(1. - np.sin(np.radians(np.pi / 2 - dlat[0])))

    if np.abs(lat[-1]) > 89.9999:
        w[-1] = np.abs(1. - np.sin(np.radians(np.pi / 2 - dlat[0])))

    return w

In [ ]:
#Also from Matt Long:
def compute_grid_area(ds, check_total=True):
    """Compute the area of grid cells."""
   
    radius_earth = 6.37122e6 # m, radius of Earth
    area_earth = 4.0 * np.pi * radius_earth**2 # area of earth [m^2]e
   
    lon_name = infer_lon_name(ds)      
    lat_name = infer_lat_name(ds)        
   
    weights = lat_weights_regular_grid(ds[lat_name])
    area = weights + 0.0 * ds[lon_name] # add 'lon' dimension
    area = (area_earth / area.sum(dim=(lat_name, lon_name))) * area
   
    if check_total:
        np.testing.assert_approx_equal(np.sum(area), area_earth)
       
    return xr.DataArray(area, dims=(lat_name, lon_name), attrs={'units': 'm^2', 'long_name': 'area'})

### Some examples from Charlie

In [ ]:
def get_gridcell_edges(lat, lon):
    IM = lon.shape[0]
    JM = lat.shape[0]
    #
    lat_edges = np.zeros(JM+1)
    lon_edges = np.zeros(IM+1)
    #
    lat_edges[0] = max(lat[0] - (lat[1] - lat[0]) / 2., -90.)
    lat_edges[JM] = min(lat[JM-1] + (lat[JM-1] - lat[JM-2]) / 2., 90.)
    for j in range(1,JM):
        lat_edges[j] = (lat[j] + lat[j-1])/2.
    #    
    lon_edges[0] = lon[0] - (lon[1] - lon[0]) / 2.
    lon_edges[IM] = lon[IM-1] + (lon[IM-1] - lon[IM-2]) / 2.
    for i in range(1,IM):
        lon_edges[i] = (lon[i] + lon[i-1])/2.
    #
    return lat_edges, lon_edges

In [ ]:
def gridcell_areas(lat, lon, mask=None, radius=6372000., lat_edges_in=False, lon_edges_in=False):

    ### assume uniform longitude spacing
    lat_edges, lon_edges = get_gridcell_edges(lat, lon)
    if lon_edges_in:
        IM = lon.shape[0] - 1
        lon_edges = lon[:]
        res_lon = lon_edges[1] - lon_edges[0]
    else:
        IM = lon.shape[0]
        res_lon = lon[1]-lon[0]
        #
    if lat_edges_in:
        JM = lat.shape[0] - 1       
        lat_edges = lat[:] 
    else:
        JM = lat.shape[0]
    southern_edge = np.fmax(lat_edges[0:JM], np.ones(JM)*-89.99)
    northern_edge = np.fmin(lat_edges[1:], np.ones(JM)*89.99)
    #
    area = Ngl.gc_qarea(southern_edge,np.zeros(JM)-res_lon/2.,\
                        northern_edge,np.zeros(JM)-res_lon/2.,\
                        northern_edge,np.zeros(JM)+res_lon/2.,\
                        southern_edge,np.zeros(JM)+res_lon/2.,\
                        radius=radius) ### 1-D array in meters sq.
    area_array = np.array(np.reshape(np.repeat(area,IM), [JM,IM]))
    if not mask==None:
        area_array = np.ma.array(area_array, mask=mask)
    return area_array

In [ ]:
def find_closest(lats, lons, lat, lon, mask=None):
    mindist = 1e20
    if mask == None:
        mask = np.zeros([len(lats), len(lons)], dtype=np.bool)
    for i in range(len(lons)):
        for j in range(len(lats)):
            if not mask[j,i]:
                dist = Ngl.gc_dist(lons[i], lats[j], lon, lat)
                if dist < mindist:
                    closest_i = i
                    closest_j = j
                    mindist = dist
    return closest_i, closest_j

In [11]:
ml_nbp = {}
bar_nbp = {}
nh_nbp = {}
for sim_name, data in co2_ds.items():
    #may want to consider also selecting the level here, but not sure correct level for mauna loa. 
    ml_nbp[sim_name] = data.sel(lat=19.5,lon=204.4, method='nearest')
    bar_nbp[sim_name] = data.sel(lat=71.3,lon=203.4, method='nearest')
    nh_nbp[sim_name]  = data.sel(lat=slice(30,90))
    

In [ ]:
print(ml_nbp.keys())
print(ml_nbp)

### Change in CO2 amplitude

In [27]:
#Barrow
barrow_pct_change = {}

for sim_name, data in bar_nbp.items():
    bar_co2_ds_late = (data.isel(plev=18, time=slice(163,165))).mean(dim=["time"])
    bar_co2_ds_early = (data.isel(plev=18, time=slice(107,109))).mean(dim=["time"])
    barrow_pct_change[sim_name] = ((bar_co2_ds_late - bar_co2_ds_early)/bar_co2_ds_early)*100

In [29]:
print(barrow_pct_change.keys())

dict_keys(['CMIP.MRI.MRI-ESM2-0.historical.Amon.gn', 'CMIP.NOAA-GFDL.GFDL-ESM4.historical.Amon.gr1', 'CMIP.BCC.BCC-ESM1.historical.Amon.gn', 'CMIP.NCAR.CESM2.historical.Amon.gn'])


In [12]:
#N. Hemisphere (45-90N)
late_change = {}
pct_change = {}

for sim_name, data in co2_ds.items():
    co2_ds_late = (data.isel(plev=18, lat=slice(45,90), time=slice(163,165))).mean(dim=["time"])
    co2_ds_early = (data.isel(plev=18, lat=slice(45,90), time=slice(107,109))).mean(dim=["time"])
    late_change[sim_name] = co2_ds_late
    pct_change[sim_name] = ((co2_ds_late - co2_ds_early)/co2_ds_early)*100
    

In [26]:
print(pct_change.keys())
print(pct_change.values())
#Note sure why I can't access a specific key...
#print(pct_change.key['CMIP.MRI.MRI-ESM2-0.historical.Amon.gn'])
#print(pct_change.key['CMIP.NOAA-GFDL.GFDL-ESM4.historical.Amon.gr1'])

dict_keys(['CMIP.MRI.MRI-ESM2-0.historical.Amon.gn', 'CMIP.NOAA-GFDL.GFDL-ESM4.historical.Amon.gr1', 'CMIP.BCC.BCC-ESM1.historical.Amon.gn', 'CMIP.NCAR.CESM2.historical.Amon.gn'])
dict_values([<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 19, lon: 128, member_id: 1)
Coordinates:
  * lat        (lat) float64 37.67 40.46 43.25 46.05 ... 79.53 82.31 85.1 87.86
  * lon        (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(128, 2), meta=np.ndarray>
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(19, 2), meta=np.ndarray>
    plev       float64 100.0
  * member_id  (member_id) <U8 'r1i1p1f1'
Dimensions without coordinates: bnds
Data variables:
    co2        (member_id, lat, lon) float32 dask.array<chunksize=(1, 19, 128), meta=np.ndarray>, <xarray.Dataset>
Dimensions:    (bnds: 2, lat: 45, lon: 288, member_id: 1)
Coordinates:
  * lat        (lat) float64 -44.5 -43.5 -42.5 -41.5 ... -3.5 -2.5 -1.5 -0.5
  * lon 

### Subsetting atmosphere level and last 20 years
Global data

In [ ]:
for sim_name, data in co2_ds.items():
    co2_ds_lev = data.isel(plev=18, time=slice(145,165))
#    print(co2_ds_lev)
    co2_ds_mean = co2_ds_lev.mean(dim=["time"])
#    print(co2_ds_mean)
    co2_ds[sim_name] = co2_ds_mean

In [ ]:
print(co2_ds.items())

In [ ]:
# weighted means not plotting, so attempting to calculate unweighted global means
#unwtd_co2_ds = {}
#for sim_name, data in co2_ds.items():
#    co2_ds_globavg = data.mean(dim=('lat','lon'))
#    print(co2_ds_globavg)
#    unwtd_co2_ds[sim_name] = co2_ds_globavg
    
# Pick lat & lon -- example from Will
#tlat = 60  #60, 70 #-5  #46  #-3 
#tlon = 240 #240,30 #300 #262 #280 
#svdGPP.climatology.sel(lat=tlat, lon=tlon, method='nearest').plot(color='g',lw=3)

#will has a loop to plot over points

#To get NH value, try:
#.sel(lat=slice(30:90),lon=slice())
#print(unwtd_co2_ds.max())

In [ ]:
#Haven't gotten any of these to work properly...
#for sim_name, data in co2_ds.items():
#    wtd_co2 = compute_grid_area(data)
#    wtd_co2 = gridcell_areas(data.lat, data.lon)

In [ ]:
wtd_co2_ds = {}
#cosine function from Katie
# note that she showed this isn't as good of a weighting metric
# using this for now because I can't figure out how to use Matt's weighting functions
#for sim_name, data in co2_ds.items():
#    lats = data.lat
#    cwt = np.cos(lats * np.pi / 180.)
#    wtd_co2 = esmlab.statistics.weighted_mean(data, dim=['lat'], weights=cwt).mean().values
#    print(wtd_co2)
#    wtd_co2_ds[sim_name] = wtd_co2

#print(wtd_co2_ds)
#why are these all float64???


#Another example from Katie -- area element weighting
## we know already that the spacing of the points is 4 degrees latitude, 5 degrees longitude
R = 6.37e6
for sim_name, data in co2_ds.items():
    nlat = data.lat
    nlon = data.lon
    dϕ = np.deg2rad(180./nlat) 
    dλ = np.deg2rad(360./nlon)
    dA = R**2 * dϕ * dλ * np.cos(np.deg2rad(data.lat))
    pixel_area = dA.where(data.notnull())
    total_land_area = pixel_area.sum(dim=['lat', 'lon'])
    ae_wtd_co2amp = ((data * pixel_area).sum(dim=['lat', 'lon']) / total_land_area).values
    wtd_co2_ds[sim_name] = ae_wtd_co2amp
print(wtd_co2_ds)

In [ ]:
co2_globavg_ds = {}
for sim_name, data in co2_ds.items():
    glob_avg = data.mean("lat","lon")
    co2_globavg_ds[sim_name] = glob_avg

In [ ]:
#maybe the weighting already calculated the means... so this might not be necessary; it doesn't work at least
#for sim_name, data in wtd_co2_ds.items():
#    global_avg = data.mean(dims=["lat","lon"])

#### Not sure how to properly work with above weights -- looks like it give a float64 value, not sure why

### Averaging ensemble means (for when I have multiple members)
#### Borrowed from Katie's code

In [ ]:
#Below code doesn't work
ens_mean_co2amp = {}
for sim_name, data in wtd_co2_ds.items():
#note that mcount doesn't work -- 'function' object has no attribute 'sizes'. Not sure how to solve
    mcount = data.sizes['member_id']  # count of ensemble members
    ens_mean_co2amp[sim_name + f' (n={mcount})'] = data.mean(dim='member_id', keep_attrs=True).resample(time='AS').mean(keep_attrs=True)
#adding var name doesn't seem to solve the problem
#    mcount = data.co2.sizes['member_id']  # count of ensemble members
#    ens_mean_co2amp[sim_name + f' (n={1})'] = data.co2.mean(dim='member_id', keep_attrs=True).resample(time='AS').mean(keep_attrs=True)

In [ ]:
#Also borrowed from Katie
#need to have above code block working to use this
# combine all the members into 1 DataArray
#members = xr.DataArray(list(ens_mean_co2amp.keys()), dims='members', name='members')
#da = xr.concat(ens_mean_co2amp.values(), dim=members)

In [ ]:
# fig 1 - using built-in xarray plotting
#fig = plt.figure(figsize=(12, 8))
#da.sel(time=slice('2015', '2100')).plot.line(x='time')
#plt.title('Projected Changes in Annual Total NBP', fontdict={'size':16})
#plt.ylabel('NBP'+' (' + da.attrs['units'] +')');


#can't figure out why I can't plot this... 
#attempting plot below
names = list(wtd_co2_ds.keys())
values = list(wtd_co2_ds.values())
print(len(names))
print(values)
x = np.arange(len(co2_globavg_ds))

print(type(names))
print(type(values))
print(type(x))

print(values)

fig = plt.figure(figsize=(12, 8))
#plt.bar(range(len(co2_globavg_ds)),values,tick_label=names)
#bar_plot = plt.bar(x,values)
#plt.title("Average NBP Amplitude for CMIP6 Models (1995-2014)")

#plt.ylabel("2m air temperature ($\degree$C)")
#plt.xticks(x, ('arwt sum','arwt max', 'arwt no norm', 'area no lf', 'cos lat', 'area element', 'area element lf', 'geocat'))

In [ ]:
#plotting based on Katie's example
#fig, ax = plt.subplots(figsize=(12,6))
#means = [TSA_glob_mean_arwt_sum, TSA_glob_mean_arwt_max, TSA_glob_mean_arwt, TSA_glob_mean_arwt_nolf, TSA_glob_mean_cwt, TSA_glob_mean_ae, TSA_glob_mean_ae_lf, TSA_glob_mean_geocat]
#x = np.arange(len(means))
#means_c = [m - 273.15 for m in means]

# color code bars based on value - though hard to distinguish close values
#from matplotlib import cm
#color_vals = np.array(means_c)
#colors = cm.hsv(color_vals / float(max(color_vals)))

#bar_plot = plt.bar(x,means_c,color = colors)
#plt.title("Comparing Global Mean Methods")
#plt.ylabel("2m air temperature ($\degree$C)")
#plt.xticks(x, ('arwt sum','arwt max', 'arwt no norm', 'area no lf', 'cos lat', 'area element', 'area element lf', 'geocat'))

# add text labels
#labels_plot = [round(m, 5) for m in means_c] # round to 4 decimal places
#def autolabel(rects):
#    for idx,rect in enumerate(bar_plot):
#        height = rect.get_height()
#        ax.text(rect.get_x() + rect.get_width()/2., 1.02*height,
#                labels_plot[idx],
#                ha='center', va='bottom', rotation=0)

#autolabel(bar_plot)

#plt.ylim(0,13);

### To Do next
 - atm level for CO2 (looks like all are standardized at 19 levels)
 - decide what region (point, global, NH) to get data for, and figure out how to do the appropriate weighting
     - note: if just choosing 30N, perhaps the lat/lon weighting Katie tried is enough -- I don't need the 
 - what kind of figure to make? Annual cycle? or just a value?
 Maybe start with plotting annual cycle for all CMIP6 models N of 30N

In [ ]:
val  = co2_ds.get('CMIP.MRI.MRI-ESM2-0.historical.fx.gn')
print(val)